In [ ]:
import requests
import json
import time
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
api_key_private= "your_api_key"
api_key_public= "your_api_key"

In [ ]:
import requests
import json
import time
import pandas as pd
from datetime import datetime, timedelta


class FootTrafficAnalyzer:
    def __init__(self):
        self.api_key_private = "your_api_key"
        self.api_key_public = "your_api_key"

    def get_all_foot_traffic(self, venue_query: str):
        """
        Searches for a venue, waits for the analysis to complete,
        and returns the live foot traffic forecast and the weekly foot traffic forecast.
        """
        print(f"Searching for venue: '{venue_query}'...")

        # Step 1: Search for the venue and start the forecast job
        search_url = "https://besttime.app/api/v1/venues/search"
        search_params = {
            'api_key_private': self.api_key_private,
            'q': venue_query,
            'num': 1,
            'fast': True,
            'format': 'raw'
        }
        search_response = requests.request("POST", search_url, params=search_params)
        search_result = search_response.json()

        if search_result.get("status") != "OK":
            print("Error during venue search.")
            return search_result

        job_id = search_result.get("job_id")
        collection_id = search_result.get("collection_id")
        print(f"Search job started with Job ID: {job_id}")

        # Step 2: Poll the progress endpoint until the job is finished
        progress_url = "https://besttime.app/api/v1/venues/progress"
        progress_params = {
            'job_id': job_id,
            'collection_id': collection_id,
            'format': 'raw'
        }

        print("Waiting for forecast analysis to complete...")
        while True:
            progress_response = requests.request("GET", progress_url, params=progress_params)
            progress_result = progress_response.json()

            if progress_result.get("job_finished"):
                print("Analysis complete.")
                break
            else:
                time.sleep(5)

        # Extract venue details and weekly forecast from the completed job
        venue_details = progress_result.get("venues", [])[0]
        venue_name = venue_details.get("venue_name")
        venue_address = venue_details.get("venue_address")
        venue_lat = venue_details.get("venue_lat")
        venue_lon = venue_details.get("venue_lon")
        weekly_foot_traffic_forecast = venue_details.get("venue_foot_traffic_forecast")

        # Step 3: Fetch the live forecast using the retrieved venue details
        print(f"Fetching live forecast for '{venue_name}'...")
        live_forecast_url = "https://besttime.app/api/v1/forecasts/live"
        live_forecast_params = {
            'api_key_private': self.api_key_private,
            'venue_name': venue_name,
            'venue_address': venue_address
        }
        live_response = requests.request("POST", live_forecast_url, params=live_forecast_params)
        live_result = live_response.json()

        # Combine the results and return them
        combined_result = {
            "live_forecast": live_result,
            "weekly_foot_traffic_forecast": weekly_foot_traffic_forecast,
            "venue_name": venue_name,
            "venue_lat": venue_lat,
            "venue_lon": venue_lon
        }

        return combined_result

    def extract_weekly_foot_traffic(self, output: dict) -> list:
        """Extract weekly foot traffic data from API response"""
        try:
            weekly_data = output["weekly_foot_traffic_forecast"]
            lat = output["venue_lat"]
            lon = output["venue_lon"]

            return [
                {
                    "day_text": day["day_info"]["day_text"],
                    "venue_open": day["day_info"]["venue_open"],
                    "venue_closed": day["day_info"]["venue_closed"],
                    "day_raw": day["day_raw"],
                    "lat": lat,
                    "long": lon
                }
                for day in weekly_data
            ]
        except KeyError as e:
            print(f"Key error while extracting weekly traffic: {e}")
            return []

    def get_weather_each(self, latitude: float, longitude: float, target_date: str, target_time_hour: int):
        """
        Fetches weather data for a specific location and time.
        """
        api_url = "https://api.open-meteo.com/v1/forecast"

        params = {
            "latitude": latitude,
            "longitude": longitude,
            "start_date": target_date,
            "end_date": target_date,
            "hourly": "temperature_2m,relativehumidity_2m,windspeed_10m",
            "timezone": "auto"
        }

        try:
            response = requests.get(api_url, params=params)
            response.raise_for_status()
            weather_data = response.json()

            hourly_df = pd.DataFrame(weather_data['hourly'])
            hourly_df['time'] = pd.to_datetime(hourly_df['time'])

            target_datetime = datetime.strptime(f"{target_date}T{target_time_hour:02d}:00", "%Y-%m-%dT%H:%M")
            specific_weather = hourly_df[hourly_df['time'] == target_datetime]

            if not specific_weather.empty:
                temperature = specific_weather['temperature_2m'].iloc[0]
                return float(temperature)
            else:
                return 30.0  # fallback

        except Exception as err:
            print(f"Weather API error: {err}")
            return 30.0  # fallback

    def get_weather(self, latitude: float, longitude: float, target_date: str):
        """Get 24-hour temperature data for a specific date"""
        temperatures = []
        for i in range(24):
            temp = self.get_weather_each(latitude, longitude, target_date, i)
            temperatures.append(temp)
        return temperatures

    def filter_and_augment_weekly_traffic(self, weekly_traffic, start_date, end_date):
        """
        Filters weekly traffic data based on date range, attaches actual calendar dates,
        and fetches hourly temperatures using the lat/long in each entry.
        """
        start = datetime.strptime(start_date, "%Y-%m-%d")
        end = datetime.strptime(end_date, "%Y-%m-%d")

        if start > end:
            start, end = end, start

        # Map each weekday (lowercase) to its corresponding date in the range
        date_by_day = {}
        current = start
        while current <= end:
            weekday = current.strftime("%A").lower()
            if weekday not in date_by_day:
                date_by_day[weekday] = current.strftime("%Y-%m-%d")
            current += timedelta(days=1)

        days_to_keep = set(date_by_day.keys())

        filtered = []
        for day_info in weekly_traffic:
            weekday = day_info["day_text"].lower()
            if weekday in days_to_keep:
                day_info_with_date = day_info.copy()
                date = date_by_day[weekday]
                day_info_with_date["date"] = date

                lat = day_info["lat"]
                lon = day_info["long"]

                if lat is not None and lon is not None:
                    print(f"Fetching weather for {day_info['day_text']} ({date})...")
                    day_info_with_date["temperatures"] = self.get_weather(lat, lon, date)
                else:
                    day_info_with_date["temperatures"] = [27.0] * 24  # fallback

                filtered.append(day_info_with_date)

        return filtered

    def analyze_touristic_site(self, site_name: str, outdoor: bool, hours_needed, start_date: str, end_date: str):
        """
        Main function that analyzes a touristic site and returns organized foot traffic data.

        Args:
            site_name (str): Name of the touristic site
            outdoor (bool): Whether the site is outdoor or not
            hours_needed (int): Number of hours to spend time at site
            start_date (str): Start date in 'YYYY-MM-DD' format
            end_date (str): End date in 'YYYY-MM-DD' format

        Returns:
            dict: Organized data with site info and daily traffic patterns
        """
        print(f"\n=== Analyzing {site_name} ===")
        print(f"Date range: {start_date} to {end_date}")
        print(f"Outdoor venue: {outdoor}")
        print(f"Hours needed: {hours_needed}")

        # Step 1: Get foot traffic data
        foot_traffic_data = self.get_all_foot_traffic(site_name)

        if foot_traffic_data.get("status") == "Error":
            return {"error": "Failed to fetch foot traffic data", "details": foot_traffic_data}

        # Step 2: Extract weekly traffic data
        weekly_traffic = self.extract_weekly_foot_traffic(foot_traffic_data)

        if not weekly_traffic:
            return {"error": "Failed to extract weekly traffic data"}

        # Step 3: Filter by date range and add weather data
        filtered_traffic = self.filter_and_augment_weekly_traffic(weekly_traffic, start_date, end_date)

        # Step 4: Organize final result
        result = {
            "name": foot_traffic_data.get("venue_name", site_name),
            "lat": foot_traffic_data.get("venue_lat"),
            "lon": foot_traffic_data.get("venue_lon"),
            "outdoor": outdoor,
            "hours_needed": hours_needed,
            "days": filtered_traffic
        }

        print(f"\n=== Analysis complete for {result['name']} ===")
        print(f"Found data for {len(filtered_traffic)} days")

        return result

In [ ]:
analyzer = FootTrafficAnalyzer()

# Test the function
result = analyzer.analyze_touristic_site(
    site_name="Giza Necropolis",
    outdoor=False,
    hours_needed=3,
    start_date="2025-08-03",
    end_date="2025-08-05"
)

print("\n=== Final Result ===")
print(json.dumps(result, indent=2))


=== Analyzing Giza Necropolis ===
Date range: 2025-08-03 to 2025-08-05
Outdoor venue: False
Hours needed: 3
Searching for venue: 'Giza Necropolis'...
Search job started with Job ID: 0645ee08-6b41-4dcf-b623-72e5715e7331
Waiting for forecast analysis to complete...
Analysis complete.
Fetching live forecast for 'Giza Necropolis'...
Fetching weather for Monday (2025-08-04)...
Fetching weather for Tuesday (2025-08-05)...
Fetching weather for Sunday (2025-08-03)...

=== Analysis complete for Giza Necropolis ===
Found data for 3 days

=== Final Result ===
{
  "name": "Giza Necropolis",
  "lat": 29.9772962,
  "lon": 31.1324955,
  "outdoor": false,
  "hours_needed": 3,
  "days": [
    {
      "day_text": "Monday",
      "venue_open": 6,
      "venue_closed": 16,
      "day_raw": [
        5,
        15,
        25,
        40,
        55,
        65,
        65,
        65,
        65,
        60,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0